In [5]:
# set up a GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"

In [6]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from datautils import GenDataset
from networkmodels import Network2, load_network, save_network, parameters_to_csv

In [7]:
# test GPU working (output should be "Using -- cuda")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print( "Using --", device )

Using -- cuda


In [8]:
# load / build data set 
filename = '../sim-data/population-sample.csv'

pred_vars_to_use = ["Age", "Sex", "MDD_hx", "Rx_hx", "Py_hx", "Sev", "Dur"]
target_var_to_use = ["Offer_Rx"]

# instantiate the data loader for the actual data
test_data = GenDataset( filename, pred_vars_to_use, target_var_to_use, trainflag = False )
test_loader = DataLoader(test_data, batch_size= len( test_data), shuffle = False, num_workers=2 )

# establish input/output dimensions
test_x, test_y = next(iter(test_loader))

input_size = test_x[0].shape[0]
output_size = test_y[0].shape[0]

print(f" Data Input size = {input_size}")
print(f" Data Target size = {output_size}")

 Data Input size = 7
 Data Target size = 1


In [9]:
hidden_size = 32
# instantiate network
model = Network2( input_size, hidden_size, output_size)

# restore state from epoch 29
load_network( model, 'big-net-state_epoch29.pth')
print(f"Model Activations Total = {model.total_activations}")


Model Activations Total = 33


In [10]:
dataset = test_loader.dataset
this_input = dataset[1][0]
this_label = dataset[1][1]

acts = np.zeros( shape = ( len(dataset), model.total_activations ) )
sample_data = np.zeros( shape = ( len(dataset), len( this_input ) ) )
sample_target = np.zeros( shape = ( len(dataset), 1 ) )

for i in range(len( dataset ) ):
    this_input = dataset[i][0]
    this_label = dataset[i][1]
    activations_vector = model.get_activations( this_input )
    acts[i,:] = activations_vector
    sample_data[i,:] = this_input
    sample_target[i,:] = this_label 

df = pd.DataFrame( np.hstack([sample_data, acts, sample_target]) )
df.to_csv('../out-data/test-acts-big-model.csv', header = False)

# dump params to a csv
parameters_to_csv( model, 'big-net-weights' )
